In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
# Programa que permite cargar una imagen y mostrarla en pantalla con los cambios de color basados en el tipo de Daltonismo del usuario.

# Variables globales
tipo_daltonismo = 0
imagen_cargada = False

# Funciones
def Deuteranopia(imagen_arreglo):
    global tipo_daltonismo, imagen_cargada
    tipo_daltonismo = 1

    # Habilita el botón de "Mostrar Imagen"
    if imagen_cargada:
        boton_Mostrar_Imagen.config(state="normal")

def Deuteranomalia(imagen_arreglo):
    global tipo_daltonismo, imagen_cargada
    tipo_daltonismo = 2

    # Habilita el botón de "Mostrar Imagen"
    if imagen_cargada:
        boton_Mostrar_Imagen.config(state="normal")
    
def Protanopia(imagen_arreglo):
    global tipo_daltonismo, imagen_cargada
    tipo_daltonismo = 3

    # Habilita el botón de "Mostrar Imagen"
    if imagen_cargada:
        boton_Mostrar_Imagen.config(state="normal")

def Protanomalia(imagen_arreglo):
    global tipo_daltonismo, imagen_cargada
    tipo_daltonismo = 4

    # Habilita el botón de "Mostrar Imagen"
    if imagen_cargada:
        boton_Mostrar_Imagen.config(state="normal")

def Tritanopia(imagen_arreglo):
    global tipo_daltonismo, imagen_cargada
    tipo_daltonismo = 5

    # Habilita el botón de "Mostrar Imagen"
    if imagen_cargada:
        boton_Mostrar_Imagen.config(state="normal")

def Tritanomalia(imagen_arreglo):
    global tipo_daltonismo, imagen_cargada
    tipo_daltonismo = 6

    # Habilita el botón de "Mostrar Imagen"
    if imagen_cargada:
        boton_Mostrar_Imagen.config(state="normal")

def TransformarDaltonismo(imagen_arreglo):
    ventana.withdraw()
    nueva_ventana = tk.Toplevel()  # Crea una nueva ventana
    nueva_ventana.title("Imagen Editada")
    nueva_ventana.configure(bg="#FFFFFF")
    nueva_ventana.geometry("800x600")  # Establece un tamaño para la nueva 
    
    titulo = tk.Label(nueva_ventana, text="IMAGEN AJUSTADA AL TIPO DE DALTONISMO", font=("Impact", 20), bg="#FFFFFF", fg="#34495E")
    titulo.pack()

    # Convierte la imagen al tipo de daltonismo seleccionado //////// AQI SE HACE LA MAGIA

    # Convierte la imagen para el daltonimo: Deuteranopia
    if tipo_daltonismo == 1:
        imagen_modificada = imagen_arreglo

    # Convierte la imagen para el daltonimo: Deuteranomalia
    elif tipo_daltonismo == 2:
        imagen_modificada = imagen_arreglo

    # Convierte la imagen para el daltonimo: Protanopia
    elif tipo_daltonismo == 3:
        imagen_modificada = imagen_arreglo

    # Convierte la imagen para el daltonimo: Protanomalia
    elif tipo_daltonismo == 4:
        imagen_modificada = imagen_arreglo

    # Convierte la imagen para el daltonimo: Tritanopia
    elif tipo_daltonismo == 5:
        imagen_modificada = imagen_arreglo

    # Convierte la imagen para el daltonimo: Tritanomalia
    elif tipo_daltonismo == 6:
        imagen_modificada = imagen_arreglo
    
    # Si no se selecciona un tipo de daltonismo, se muestra la imagen original. Uno nunca se sabe
    else:
        imagen_modificada = imagen_arreglo

    # Convierte la imagen en un objeto Pillow (PIL)
    imagen_pil = Image.fromarray(imagen_modificada)

    # Define el tamaño máximo en píxeles (cambiar según tus necesidades)
    ancho_maximo = 500
    alto_maximo = 300

    # Redimensiona la imagen si es necesario
    if imagen_pil.width > ancho_maximo or imagen_pil.height > alto_maximo:
        imagen_pil.thumbnail((ancho_maximo, alto_maximo))

    # Convierte la imagen de Pillow a un objeto Tkinter PhotoImage
    imagen_tk_umbrealizacion = ImageTk.PhotoImage(imagen_pil)

    # Muestra la imagen en una etiqueta
    etiqueta_imagen_umbrealizada = tk.Label(nueva_ventana, image=imagen_tk_umbrealizacion)
    etiqueta_imagen_umbrealizada.image = imagen_tk_umbrealizacion  # ¡Importante! Mantener una referencia para evitar que se borre la imagen
    etiqueta_imagen_umbrealizada.pack()
    # Frame para botones
    frame_botones = tk.Frame(nueva_ventana, bg="#FFFFFF")
    frame_botones.pack(pady=10)

    # Botón Regresar
    boton_cerrar = tk.Button(frame_botones, text="Regresar", command=lambda: cerrar_ventana(nueva_ventana))
    boton_cerrar.pack(side="left", padx=10)

    # Botón Guardar
    boton_guardar = tk.Button(frame_botones, text="Guardar", command=lambda: guardar_imagen(imagen_modificada))
    boton_guardar.pack(side="left", padx=10)

def cerrar_ventana(nueva_ventana):
    # Cerrar la ventana actual y mostrar la ventana principal
    nueva_ventana.destroy()
    ventana.deiconify()
def guardar_imagen(imagen_guardar):
    cv.imwrite("imagen_editada.png",imagen_guardar)
def cargar_imagen():
    global imagen_arreglo, tipo_daltonismo, imagen_cargada
    ruta_imagen = filedialog.askopenfilename(title="Selecciona una imagen", filetypes=[("Archivos de imagen", "*.png *.jpg *.jpeg *.gif *.bmp")])
    if ruta_imagen:
        imagen_cargada = True
        # Abre la imagen seleccionada con Pillow (PIL)
        imagen = Image.open(ruta_imagen)
        imagen_arreglo = np.array(imagen)

        # Define el tamaño máximo en píxeles (cambiar según tus necesidades)
        ancho_maximo = 500
        alto_maximo = 300

        # Redimensiona la imagen si es necesario
        if imagen.width > ancho_maximo or imagen.height > alto_maximo:
            imagen.thumbnail((ancho_maximo, alto_maximo))

        # Convierte la imagen en un objeto Tkinter PhotoImage
        imagen_tk = ImageTk.PhotoImage(imagen)

        # Muestra la imagen en una etiqueta cliclable
        etiqueta_imagen.config(image=imagen_tk, cursor="hand2")
        etiqueta_imagen.image = imagen_tk  # ¡Importante! Mantener una referencia para evitar que se borre la imagen
        etiqueta_imagen.bind("<Button-1>", lambda event: cargar_imagen())  # Asocia la función cargar_imagen al clic

        # Deshabilita el botón de "Cargar Imagen" después de cargar la imagen
        boton_cargar_imagen.pack_forget()

        # Habilitar los botones de tipo de daltonismo
        boton_deuteranopia.config(state="normal")
        boton_deuteranomalia.config(state="normal")
        boton_protanopia.config(state="normal")
        boton_protanomalia.config(state="normal")
        boton_tritanopia.config(state="normal")
        boton_tritanomalia.config(state="normal")

        

ventana = tk.Tk()
ventana.title("Ventana Principal")
ventana.configure(bg="#FFFFFF")
ventana.geometry("800x600")

titulo = tk.Label(ventana, text="ASISTENCIA PARA DALTONISMO", font=("Impact", 30), bg="#FFFFFF", fg="#34495E")
titulo.pack()

etiqueta_imagen = tk.Label(ventana)
etiqueta_imagen.pack()

# Crea un botón de "Cargar Imagen"
boton_cargar_imagen = tk.Button(ventana, text="Cargar Imagen", font=("Lucida Sans Unicode", 15), bg="#FFFFFF", fg="#34495E", command=cargar_imagen, width=30, height=4)
boton_cargar_imagen.pack()

# Crea label para introducir el tipo de daltonismo:
label_daltonismo = tk.Label(ventana, text="DALTONISMO", font=("Impact", 15), bg="#FFFFFF", fg="#34495E")
label_daltonismo.pack()

# Crea un marco para los botones de procesamiento de imagen
marco_botones = tk.Frame(ventana, bg="#FFFFFF")
marco_botones.pack(pady=10)

# Crea los botones de tipo de daltonismo
boton_deuteranopia = tk.Button(marco_botones, text="Deuteranopia", font=("Lucida Sans Unicode", 8), bg="#FFFFFF", fg="#34495E", command=lambda: Deuteranopia(imagen_arreglo), state="disabled", width=15, height=2)
boton_deuteranopia.pack(side="left", padx=10)
boton_deuteranomalia = tk.Button(marco_botones, text="Deuteranomalia", font=("Lucida Sans Unicode", 8), bg="#FFFFFF", fg="#34495E", command=lambda: Deuteranomalia(imagen_arreglo), state="disabled", width=15, height=2)
boton_deuteranomalia.pack(side="left", padx=10)
boton_protanopia = tk.Button(marco_botones, text="Protanopia", font=("Lucida Sans Unicode", 8), bg="#FFFFFF", fg="#34495E", command=lambda: Protanopia(imagen_arreglo), state="disabled", width=15, height=2)
boton_protanopia.pack(side="left", padx=10)
boton_protanomalia = tk.Button(marco_botones, text="Protanomalia", font=("Lucida Sans Unicode", 8), bg="#FFFFFF", fg="#34495E", command=lambda: Protanomalia(imagen_arreglo), state="disabled", width=15, height=2)
boton_protanomalia.pack(side="left", padx=10)
boton_tritanopia = tk.Button(marco_botones, text="Tritanopia", font=("Lucida Sans Unicode", 8), bg="#FFFFFF", fg="#34495E", command=lambda: Tritanopia(imagen_arreglo), state="disabled", width=15, height=2)
boton_tritanopia.pack(side="left", padx=10)
boton_tritanomalia = tk.Button(marco_botones, text="Tritanomalia", font=("Lucida Sans Unicode", 8), bg="#FFFFFF", fg="#34495E", command=lambda: Tritanomalia(imagen_arreglo), state="disabled", width=15, height=2)
boton_tritanomalia.pack(side="left", padx=10)


# Crea un marco para los botones de procesamiento de imagen
marco_botones = tk.Frame(ventana, bg="#FFFFFF")
marco_botones.pack(pady=10)

# Crea el botón de "Mostrar Imagen" inicialmente deshabilitado
boton_Mostrar_Imagen = tk.Button(marco_botones, text="Mostrar Imagen", font=("Lucida Sans Unicode", 15), bg="#FFFFFF", fg="#34495E", command=lambda: TransformarDaltonismo(imagen_arreglo), state="disabled", width=15, height=2)
boton_Mostrar_Imagen.grid(row=0, column=0, padx=10, pady=10)


ventana.mainloop()